### 이번주 과제는 DACON의 구내 식당 식수인원 예측 AI 경진대회의 데이터를 그대로 사용하여 직접 모델링 하고 제출까지 해보는 과제입니다! 

Decision Tree의 파라미터 튜닝 외에도 이전 교육세션에서 배우신 Feature-engineering, 다른 회귀 모델도 자율적으로 사용하시는걸 모두 허용합니다 :) 처음 하시는 분들도 아 이런식으로 데이터 공모전에 참여하는구나의 감만 익혀가시는 걸 목표로 하시면 좋을 것 같습니다. 과제하시면서 질문이 있으신 분들은 언제든 질문방 or 개인톡으로 연락주세요 :)

In [15]:
import pandas as pd
import numpy as np

### 데이터셋 로드

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [18]:
print(f'{train.shape = }')
print(f'{test.shape  = }')

train.shape = (1205, 12)
test.shape  = (50, 10)


In [19]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [20]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [21]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [23]:
train.describe()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,2807.815768,157.913693,241.142739,274.117012,43.506224,890.334440,461.772614
std,171.264404,144.190572,43.532298,246.239651,109.937400,209.505057,139.179202
min,2601.000000,23.000000,41.000000,0.000000,0.000000,296.000000,0.000000
25%,2645.000000,71.000000,217.000000,4.000000,0.000000,758.000000,406.000000
50%,2760.000000,105.000000,245.000000,299.000000,0.000000,879.000000,483.000000
75%,2962.000000,185.000000,272.000000,452.000000,0.000000,1032.000000,545.000000
max,3305.000000,1224.000000,378.000000,1044.000000,533.000000,1459.000000,905.000000


### 요일을 숫자로 매핑

In [24]:
train.요일.unique()

array(['월', '화', '수', '목', '금'], dtype=object)

In [25]:
train['요일'] = train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
test['요일'] = test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

### 수치 데이터만 feature로 선택

In [26]:
x_train = train[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]
y1_train = train['중식계']
y2_train = train['석식계']

x_test = test[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]

### Decision tree model 생성

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

### 석식계 예측모델과 중식계 예측모델 Train test split

In [47]:
x_train_lunch, x_val_lunch, y_train_lunch, y_val_lunch = train_test_split(x_train, y1_train, test_size=0.2, random_state=42) 
x_train_dinner, x_val_dinner, y_train_dinner, y_val_dinner = train_test_split(x_train, y2_train, test_size=0.2, random_state=42)

Q) validation set의 역할은 무엇인가요?

A) train set으로 모델 학습 후 학습이 잘 됐는지 판별할 때 validation set을 사용합니다. 

### 중식계 예측모델 Grid Search

In [48]:
from sklearn.model_selection import GridSearchCV
grid_dt = DecisionTreeRegressor()

# 테스트 하고 싶은 parameter 값들을 parameter 별로 sequence 구조의 데이터(list, numpy array, etc...)에 넣어줍니다.
# 이 때 변수 이름은 꼭 모델 parameter 이름과 동일할 필요는 없습니다.

max_depth = [3, 4, 5, 6, 7, 8, 9]
max_leaf = [8, 12, 13, 14, 15, 16, 20]

# 위의 세 parameter를 하나로 묶기 위해서 다음과 같은 Dictionary 형태로 만들어줍니다.
# 여기서 key 값은 str 형태로, 정확하게 모델 parameter 이름과 동일해야 합니다.

parameter_grid = {'max_depth': max_depth, 'max_leaf_nodes': max_leaf}

In [49]:
gs = GridSearchCV(estimator=grid_dt, param_grid=parameter_grid)
gs.fit(x_train_lunch, y_train_lunch)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [8, 12, 13, 14, 15, 16, 20]})

In [50]:
print(f'GridSearch 최적 parameter : {gs.best_params_}')
print(f'GridSearch 최고 Validation Score : {gs.best_score_:.3f}')

GridSearch 최적 parameter : {'max_depth': 5, 'max_leaf_nodes': 14}
GridSearch 최고 Validation Score : 0.678


### 중식계 예측모델 학습

In [51]:
# Grid Search로 찾은 최적의 파라미터 값으로 모델 생성

model1 = DecisionTreeRegressor(max_depth=6, max_leaf_nodes=14)
model1.fit(x_train_lunch, y_train_lunch)
reg_prediction1 = model1.predict(x_val_lunch)

In [52]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print(f'MAE      : {mean_absolute_error(y_val_lunch, reg_prediction1):.3f}')
print(f'MSE      : {mean_squared_error(y_val_lunch, reg_prediction1):.3f}')
print(f'r2 score : {r2_score(y_val_lunch, reg_prediction1):.3f}')

MAE      : 94.663
MSE      : 16388.690
r2 score : 0.613


### 학습된 모델로 중식계 test data 예측

Q) MAE, MSE, r2_score가 각각 의미하는 바를 간단하게 적어주세요.

A) MAE : model1을 이용해 예측한 X_val_lunch에 대한 결과값과 실제 결과값의 차이의 평균

   MSE : model1을 이용해 예측한 X_val_lunch에 대한 결과값과 실제 결과값의 차이의 제곱의 평균
   
   r^2 : model1의 X_val_lunch & y_val_lunch 데이터에 대한 적합성

In [53]:
pred1 = model1.predict(x_test)
submission['중식계'] = pred1
submission.head()

,일자,중식계,석식계
0,2021-01-27,944.707602,0
1,2021-01-28,944.707602,0
2,2021-01-29,598.504505,0
3,2021-02-01,1171.808511,0
4,2021-02-02,944.707602,0


### 석식계 예측모델 Grid Search

In [54]:
gs.fit(x_train_dinner, y_train_dinner)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [8, 12, 13, 14, 15, 16, 20]})

In [55]:
print(f'GridSearch 최적 parameter : {gs.best_params_}')
print(f'GridSearch 최고 Validation Score : {gs.best_score_:.3f}')

GridSearch 최적 parameter : {'max_depth': 4, 'max_leaf_nodes': 16}
GridSearch 최고 Validation Score : 0.343


### 석식계 예측모델 학습

In [56]:
## 중식계 예측모델 학습코드를 참고하여 직접 작성해보세요!

model2 = DecisionTreeRegressor(max_depth=4, max_leaf_nodes=16)
model2.fit(x_train_dinner, y_train_dinner)
reg_prediction2 = model2.predict(X_val_dinner)

In [57]:
print(f'MAE      : {mean_absolute_error(y_val_dinner, reg_prediction2):.3f}')
print(f'MSE      : {mean_squared_error(y_val_dinner, reg_prediction2):.3f}')
print(f'r2 score : {r2_score(y_val_dinner, reg_prediction2):.3f}')

MAE      : 78.808
MSE      : 12974.235
r2 score : 0.346


### 학습된 모델로 석식계 test data 예측

In [54]:
pred2 = model2.predict(x_test)
submission['석식계'] = pred2
submission.head()

,일자,중식계,석식계
0,2021-01-27,944.707602,333.788288
1,2021-01-28,944.707602,414.317073
2,2021-01-29,598.504505,333.788288
3,2021-02-01,1171.808511,414.317073
4,2021-02-02,944.707602,414.317073


### submission.csv 저장 후 제출

In [55]:
submission.to_csv('dt_submission.csv', index=False)

#### 본인의 깃헙 repository에 과제 코드와 제출한 화면 캡처본을 함께 업로드해주세요 

***

In [ ]:
from sklearn.linear_model import ARDRegression, GammaRegressor, HuberRegressor, LinearRegression, LogisticRegression, PassiveAggressiveRegressor, PoissonRegressor, RANSACRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor

models = ['ARDRegression', 'HuberRegressor', 'LinearRegression', 'LogisticRegression', 'PassiveAggressiveRegressor', 'PoissonRegressor',
          'RANSACRegressor', 'GaussianNB', 'AdaBoostRegressor', 'BaggingRegressor', 'ExtraTreesRegressor', 'GradientBoostingRegressor',
          'RandomForestRegressor']

In [84]:
for model in models:
    model = globals()[model]()
    model.fit(x_train_lunch, y_train_lunch)
    print(f'{model = }')
    print(f'MAE : {mean_absolute_error(model.predict(x_val_lunch), y_val_lunch)}')
    print()

model = ARDRegression()
MAE : 97.57541536500294

model = HuberRegressor()
MAE : 96.94272686504263

model = LinearRegression()
MAE : 97.51832254619663



C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_glm\glm.py:285: ConvergenceWar

model = LogisticRegression()
MAE : 164.00414937759336

model = PassiveAggressiveRegressor()
MAE : 370.28945006053806

model = PoissonRegressor()
MAE : 97.00361044241028

model = RANSACRegressor()
MAE : 107.58508498637903

model = GaussianNB()
MAE : 98.95020746887967

model = AdaBoostRegressor()
MAE : 90.98431772348835

model = BaggingRegressor()
MAE : 88.46804979253112

model = ExtraTreesRegressor()
MAE : 83.10697095435684

model = GradientBoostingRegressor()
MAE : 80.94280898735991

model = RandomForestRegressor()
MAE : 84.21497095435683



In [83]:
for model in models:
    model = globals()[model]()
    model.fit(x_train_dinner, y_train_dinner)
    print(f'{model = }')
    print(f'MAE : {mean_absolute_error(model.predict(x_val_dinner), y_val_dinner)}')
    print()

model = ARDRegression()
MAE : 78.12929044852673

model = HuberRegressor()
MAE : 73.17303729410933

model = LinearRegression()
MAE : 77.90455244106205



C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_glm\glm.py:285: ConvergenceWar

model = LogisticRegression()
MAE : 133.8921161825726

model = PassiveAggressiveRegressor()
MAE : 87.64587765904885

model = PoissonRegressor()
MAE : 78.41743076480157

model = RANSACRegressor()
MAE : 77.28783643930308

model = GaussianNB()
MAE : 120.68879668049793

model = AdaBoostRegressor()
MAE : 89.99028057053575

model = BaggingRegressor()
MAE : 72.84336099585062

model = ExtraTreesRegressor()
MAE : 71.18850622406639

model = GradientBoostingRegressor()
MAE : 69.16498586221331

model = RandomForestRegressor()
MAE : 68.41326763485478



In [90]:
model1 = GradientBoostingRegressor()
model1.fit(x_train, y1_train)

model2 = RandomForestRegressor()
model2.fit(x_train, y2_train)

RandomForestRegressor()

In [91]:
pred1 = model1.predict(x_test)
submission['중식계'] = pred1

pred2 = model2.predict(x_test)
submission['석식계'] = pred2

submission.head()

,일자,중식계,석식계
0,2021-01-27,1012.884646,381.05
1,2021-01-28,948.311273,388.09
2,2021-01-29,662.063320,270.02
3,2021-02-01,1184.526902,458.45
4,2021-02-02,979.543716,425.86


In [92]:
submission.to_csv('../../../DACON/구내식당 식수 인원 예측/submission.csv', index=False)